This is a simple example of SimSwap on processing video with multiple faces. You can change the codes for inference based on our other scripts for image or single face swapping.

Code path: https://github.com/neuralchen/SimSwap

Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

In [1]:
## make sure you are using a runtime with GPU
## you can check at Runtime/Change runtime type in the top bar.
!nvidia-smi

Sat Aug  2 03:25:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Installation

All file changes made by this notebook are temporary.
You can try to mount your own google drive to store files if you want.


In [13]:
!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull

Cloning into 'SimSwap'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (508/508), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 1141 (delta 418), reused 381 (delta 381), pack-reused 633 (from 1)
Receiving objects: 100% (1141/1141), 211.46 MiB | 46.31 MiB/s, done.
Resolving deltas: 100% (600/600), done.
Already up to date.


In [3]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303884 sha256=4515146e26cea5e20ffafbcdd3ef55d8a85a6ac5bcafb7d7702c2e0fd99c609d
  Stored in directory: /root/.cache/pip/wheels/1b/28/50/248b15750b57c6b163d89d265f242e9cf6bce0bedfea3120aa
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.37.0
    Uninstalling imageio-2.37.0:
      Successfully uninstalled imageio-2.37.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [4]:
import os
os.chdir("SimSwap")
!ls

 cog.yaml	       README.md
 crop_224	      'SimSwap colab.ipynb'
 data		       simswaplogo
 demo_file	       test_one_image.py
 docs		       test_video_swapmulti.py
 download-weights.sh   test_video_swap_multispecific.py
 insightface_func      test_video_swapsingle.py
 LICENSE	       test_video_swapspecific.py
 models		       test_wholeimage_swapmulti.py
 MultiSpecific.ipynb   test_wholeimage_swap_multispecific.py
 options	       test_wholeimage_swapsingle.py
 output		       test_wholeimage_swapspecific.py
 parsing_model	       train.ipynb
 pg_modules	       train.py
 predict.py	       util


In [8]:
!pip install googledrivedownloader


In [22]:
# !pip install googledrivedownloader

# from google_drive_downloader import GoogleDriveDownloader as gdd

# gdd.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar',
#                                     unzip=False)

In [27]:
!pip install googledrivedownloader


In [28]:
!pip install --upgrade --force-reinstall googledrivedownloader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: charset_normalizer
    Found existing installation: charset-normalizer 3.4.2
    Uninstalling charset-normalizer-3.4.2:
      Successfully uninstalled charset-normalizer-3.4.2
  Attempting uninstall: certifi
    Found existing installation: certifi 2025.7.14
    Uninstalling certi

In [2]:
!mkdir -p ./arcface_model/checkpoints
!tar -xf ./arcface_model/arcface_checkpoint.tar -C ./arcface_model/checkpoints


tar: ./arcface_model/arcface_checkpoint.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [12]:
!mkdir -p /content/SimSwap/arcface_model/checkpoints
!tar -xf /content/SimSwap/arcface_model/arcface_checkpoint.tar -C /content/SimSwap/arcface_model/checkpoints


tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [11]:
!file /content/SimSwap/arcface_model/arcface_checkpoint.tar


/content/SimSwap/arcface_model/arcface_checkpoint.tar: cannot open `/content/SimSwap/arcface_model/arcface_checkpoint.tar' (No such file or directory)


In [8]:
!unzip arcface_checkpoint.zip -d checkpoints


unzip:  cannot find or open arcface_checkpoint.zip, arcface_checkpoint.zip.zip or arcface_checkpoint.zip.ZIP.


In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd


ModuleNotFoundError: No module named 'google_drive_downloader'

In [26]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1TLnIdufzwesDbyr_nVTR7Zx9oRHMLhN',
                                    dest_path='./arcface_model/arcface_checkpoint.tar',
                                    unzip=False)


ModuleNotFoundError: No module named 'google_drive_downloader'

In [25]:
from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

ModuleNotFoundError: No module named 'google_drive_downloader'

In [ ]:
## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

### Now onedrive file can be downloaded in Colab directly!
### If the link blow is not permanent, you can just download it from the
### open url(can be found at [our repo]/doc/guidance/preparation.md) and copy the assigned download link here.
### many thanks to woctezuma for this very useful help
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/


## Inference

In [32]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions # Import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

In [33]:
transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

In [35]:
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.pic_a_path = './demo_file/Iron_man.jpg' ## or replace it with image from your own google drive
opt.video_path = './demo_file/multi_people_1080p.mp4' ## or replace it with video from your own google drive
opt.output_path = './output/demo.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/checkpoints/arcface_checkpoint.pth' # Updated path to the unzipped file
opt.isTrain = False
opt.use_mask = True  ## new feature up-to-date

crop_size = opt.crop_size

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

with torch.no_grad():
    pic_a = opt.pic_a_path
    # img_a = Image.open(pic_a).convert('RGB')
    img_a_whole = cv2.imread(pic_a)
    img_a_align_crop, _ = app.get(img_a_whole,crop_size)
    img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB))
    img_a = transformer_Arcface(img_a_align_crop_pil)
    img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])

    # convert numpy to tensor
    img_id = img_id.cuda()

    #create latent id
    img_id_downsample = F.interpolate(img_id, size=(112,112))
    latend_id = model.netArc(img_id_downsample)
    latend_id = latend_id.detach().to('cpu')
    latend_id = latend_id/np.linalg.norm(latend_id,axis=1,keepdims=True)
    latend_id = latend_id.to('cuda')

    video_swap(opt.video_path, latend_id, model, app, opt.output_path, temp_results_dir=opt.temp_path, use_mask=opt.use_mask)

------------ Options -------------
Arc_path: arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
crop_size: 512
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
f: /root/.local/share/jupyter/runtime/kernel-180f147b-dbbd-43f5-ace8-132898980c6e.json
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: False
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: 

FileNotFoundError: [Errno 2] No such file or directory: './arcface_model/checkpoints/arcface_checkpoint.pth'

In [11]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import zipfile
import os

zip_path = "/content/SimSwap - Copy.zip"  # Change if needed

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

print("✅ Extracted successfully.")


In [13]:
!pip install insightface onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [14]:
!git lfs install
!git clone https://huggingface.co/deepghs/insightface


Git LFS initialized.
Cloning into 'insightface'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 45 (from 1)
Unpacking objects: 100% (48/48), 2.42 MiB | 6.97 MiB/s, done.
Filtering content: 100% (20/20), 485.80 MiB | 119.93 MiB/s, done.


In [21]:
!pip install -U insightface onnxruntime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060437 sha256=3b11689507fe69cdda5d0e992670963e046f9b162b3a7246b972a2e6d726edc1
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
  Attempting uninstall: insightface
    Found existing installation: insightface 0.2.1
    Uninstalling insightface-0.2.1:
      Successfully uninstalled insightface-0.2.1


In [1]:
import insightface
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0)  # use ctx_id=-1 if you're on CPU


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:02<00:00, 101484.45KB/s]
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [3]:
from google.colab import files
uploaded = files.upload()

Saving 98437.jpg to 98437.jpg
Saving Above.mp4 to Above.mp4


In [6]:
!ls /content/


arcface_model  insightface  sample_data  SimSwap


In [8]:
%cd /content/SimSwap/arcface_model
!tar -xf arcface_checkpoint.tar


/content/SimSwap/arcface_model
tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [9]:
!file /content/SimSwap/arcface_model/arcface_checkpoint.tar


/content/SimSwap/arcface_model/arcface_checkpoint.tar: Zip archive data, at least v0.0 to extract, compression method=store


In [10]:
%cd /content/SimSwap/arcface_model
!mv arcface_checkpoint.tar arcface_checkpoint.pth


/content/SimSwap/arcface_model


In [14]:
%cd /content/SimSwap

!python test_video_swapsingle.py \
    --isTrain false \
    --name people \
    --Arc_path ./arcface_model/arcface_checkpoint.tar \
    --video_path ./demo/demo_file/Above.mp4 \
    --pic_a_path ./demo/demo_file/98437.jpg \
    --output_path ./output/ \
    --temp_path ./temp_results/


/content/SimSwap
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(

In [25]:
!tar -tvf /content/SimSwap/arcface_model/checkpoints/arcface_checkpoint.tar


tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [27]:
$ gzip -d file.tar.gz
$ tar -xvf file.tar

SyntaxError: invalid syntax (ipython-input-3625890950.py, line 1)

In [28]:
# Rename the file to reflect its actual type (zip)
!mv /content/SimSwap/arcface_model/arcface_checkpoint.tar /content/SimSwap/arcface_model/arcface_checkpoint.zip
# Unzip the arcface checkpoint file
!unzip /content/SimSwap/arcface_model/arcface_checkpoint.zip -d /content/SimSwap/arcface_model/checkpoints/

Archive:  /content/SimSwap/arcface_model/arcface_checkpoint.zip
fatal error: read failure while seeking for End-of-centdir-64 signature.
  This zipfile is corrupt.
unzip:  cannot find zipfile directory in one of /content/SimSwap/arcface_model/arcface_checkpoint.zip or
        /content/SimSwap/arcface_model/arcface_checkpoint.zip.zip, and cannot find /content/SimSwap/arcface_model/arcface_checkpoint.zip.ZIP, period.


In [36]:
!ls /content/SimSwap/arcface_model/checkpoints/

In [37]:
%cd /content/SimSwap/arcface_model
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!unzip arcface_checkpoint.tar -d ./checkpoints/

/content/SimSwap/arcface_model
--2025-08-02 05:32:54--  https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/374891081/f01468b3-446b-4867-8c78-6d496183f9e6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-02T06%3A16%3A54Z&rscd=attachment%3B+filename%3Darcface_checkpoint.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-02T05%3A16%3A41Z&ske=2025-08-02T06%3A16%3A54Z&sks=b&skv=2018-11-09&sig=Wng5enbcGM22SdPqAu%2F7KZrY%2B6%2Fgc%2FAJHaAJIzYrcLg%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NDExMzA3NCwibmJmIjoxNzU0MTEyN